# Домашнее задание №4


Это домашнее задание можно выполнять целиком в этом ноутбуке, либо алгоритмы написать в отдельном файле и импортировать сюда, для использования. В папке data лежат два файла islands.fasta и nonIslands.fasta. В них хранятся прочтения из CpG островков и из обычных участков генома соответственно, этими данными нужно будет воспользоваться в первом задании.

In [288]:
from Bio import SeqIO
from collections import Counter, defaultdict
from itertools import product
import numpy as np

## Задача №1 (1)
Определите частоты генерации для каждого из нуклеотидов внутри CpG островков и вне их. Посчитайте так-же частоты для всех упорядоченных пар нуклеотидов и сравните частоту пары CG внутри островков и снаружи. Сделайте вывод. 

In [75]:
def seq_frequency(seq):
    len_seq = len(seq)
    
    nucl_cnt = Counter(seq)
    nucl_freq = {nucleotide: nucl_cnt[nucleotide] / len_seq for nucleotide, cnt in nucl_cnt.items()}

    pair_cnt = defaultdict(int)
    for i in range(len(seq)-1):
        pair = seq[i] + seq[i+1]
        pair_cnt[pair] += 1
    pair_freq = {pair: cnt/(len_seq-1) for pair, cnt in pair_cnt.items()}
    
    return nucl_freq, pair_freq

In [76]:
alphabet = ['A', 'T', 'G', 'C']

def file_frequency(fasta_file, alphabet):
    all_nucl = {nucl: 0 for nucl in alphabet}
    all_pairs = {''.join(pair): 0 for pair in product(alphabet, repeat=2)}
    n_seq = 0

    for record in SeqIO.parse(fasta_file, "fasta"):
        n_seq += 1
        seq = str(record.seq).upper()
        nucl_freq, pair_freq = seq_frequency(seq)
        
        for nucleotide in all_nucl:
            all_nucl[nucleotide] += nucl_freq.get(nucleotide, 0)
        for pair in all_pairs:
            all_pairs[pair] += pair_freq.get(pair, 0)

    mean_nucl_freq = {nucl: freq/n_seq for nucl, freq in all_nucl.items()}
    mean_pair_freq = {pair: freq/n_seq for pair, freq in all_pairs.items()}

    return mean_nucl_freq, mean_pair_freq

#### Определение частоты генерации для каждого из нуклеотидов внутри CpG островков и вне их

В файлах fasta хранится множество последовательностей. Изначально я посчитала частоты внутри каждой последовательности (функция seq_frequency), затем усреднила эти частоты по всем последовательностям одного файла (file_frequency). Выведенные ниже частоты - усреднённые по всем последовательностям отдельно для файлов "islands.fasta" и "nonIslands.fasta".

In [77]:
islands_nucl, islands_pair = file_frequency("./data/islands.fasta", alphabet)
print("Средние частоты нуклеотидов внутри CpG островков:")
for nucl, freq in sorted(islands_nucl.items()):
    print(f"{nucl}: {freq:.4f}")

print("\nСредние частоты пар нуклеотидов внутри CpG островков:")
for pair, freq in sorted(islands_pair.items()):
    print(f"{pair}: {freq:.4f}")

Средние частоты нуклеотидов внутри CpG островков:
A: 0.2025
C: 0.2964
G: 0.2983
T: 0.2029

Средние частоты пар нуклеотидов внутри CpG островков:
AA: 0.0512
AC: 0.0459
AG: 0.0719
AT: 0.0334
CA: 0.0629
CC: 0.1028
CG: 0.0590
CT: 0.0717
GA: 0.0599
GC: 0.0882
GG: 0.1036
GT: 0.0467
TA: 0.0285
TC: 0.0595
TG: 0.0638
TT: 0.0510


In [78]:
nonIslands_nucl, nonIslands_pair = file_frequency("./data/nonIslands.fasta", alphabet)
print("Средние частоты нуклеотидов вне CpG островков:")
for nucl, freq in sorted(nonIslands_nucl.items()):
    print(f"{nucl}: {freq:.4f}")

print("\nСредние частоты пар нуклеотидов вне CpG островков:")
for pair, freq in sorted(nonIslands_pair.items()):
    print(f"{pair}: {freq:.4f}")

Средние частоты нуклеотидов вне CpG островков:
A: 0.2893
C: 0.1961
G: 0.1984
T: 0.3161

Средние частоты пар нуклеотидов вне CpG островков:
AA: 0.0899
AC: 0.0471
AG: 0.0702
AT: 0.0821
CA: 0.0682
CC: 0.0500
CG: 0.0053
CT: 0.0727
GA: 0.0602
GC: 0.0342
GG: 0.0501
GT: 0.0539
TA: 0.0710
TC: 0.0648
TG: 0.0729
TT: 0.1075


#### Сравнение частоты пары CG внутри островков и снаружи

In [82]:
print("Частота CG внутри островков:", islands_pair['CG'])
print("Частота CG вне островков:", nonIslands_pair['CG'])

Частота CG внутри островков: 0.05898381356980459
Частота CG вне островков: 0.0052779765716222836


In [84]:
islands_pair['CG']/nonIslands_pair['CG']

11.175459528740353

Вывод: частота пары CG в CpG-островках в 11 раз выше, чем в обычных участках генома.

## Задача №2 (2)
Напишите марковскую модель, которая имеет открытые состояния {A, T, G, C}, и скрытые состояния {+, -}. Когда модель в состоянии **+**, то вероятность генерации некоторого символа нуклеотида соответствует его частоте внутри CpG островков, вычислиному в первом задании, если состояние **-**, то частоте вне островков. Вероятность остаться внутри островка 0.95, а перейти в обычный геном 0.05. Для остальной части генома соответствующие вероятности 0.995 и 0.005. Саму модель можно реализовать в виде итератора, определив метод next, который возвращает пару - состояние и нуклеотид, который в этом состоянии произведен.    
Воспользуйтесь данной моделью для того чтобы сгенерировать набор из 20 последовательностей длинной от 1 000 до 100 000, причем к каждой последовательности должна прилагаться последовательность состояний.

In [280]:
class hidden_Markov_model:
    def __init__(self, generate_len, islands_freq, nonIslands_freq):
        self.generate_len = generate_len
        self.curr_len = 0
        self.alphabet_nucleotide = list(islands_freq.keys())
        self.alphabet_states = ['+', '-']
        self.emission_probs = {self.alphabet_states[0]: list(islands_freq.values()), 
                               self.alphabet_states[1]: list(nonIslands_freq.values())}
        self.transition_probs = {self.alphabet_states[0]: [0.95, 0.05], # + -
                                 self.alphabet_states[1]: [0.005, 0.995]} # + -
        self.curr_state = np.random.choice(self.alphabet_states, p=[0.1, 0.9])
    
    def __next__(self):
        if self.curr_len >= self.generate_len:
            raise StopIteration
            
        nucleotide = np.random.choice(self.alphabet_nucleotide,
                                      p=self.emission_probs[self.curr_state])
        next_state = np.random.choice(self.alphabet_states,
                                      p=self.transition_probs[self.curr_state])
        
        output = (self.curr_state, nucleotide)
        self.curr_len += 1
        self.curr_state = next_state
        return output

    def __iter__(self):
        return self

def generate_sequence(generate_len, islands_freq, nonIslands_freq):
    generator = hidden_Markov_model(generate_len, islands_freq, nonIslands_freq)
    states, sequence = zip(*generator)
    return list(states), list(sequence)

In [281]:
filename_generated_seq = "./data/generated_HMM_sequences.txt"

In [282]:
with open(filename_generated_seq, 'w') as f:
    for generate_len in range(1000, 100001, 4950): # ровно 20 последовательностей
        states, sequence = generate_sequence(generate_len, islands_nucl, nonIslands_nucl)
        f.write(f">{len(sequence)}\n")
        f.write(f"{''.join(sequence)}\n")
        f.write(f"{''.join(states)}\n")
        f.write("\n")

Пример генерации:

In [283]:
states, sequence = generate_sequence(1000, islands_nucl, nonIslands_nucl)
print(f">{len(sequence)}")
print(f"{''.join(sequence)}")
print(f"{''.join(states)}")

>1000
GCGGGAACCGTTTATACCTCCAAATTAGTTAAATAAGTCACAACGGATATGTAATGGAATATCTCATATATTTCCGGCCTCCTTGAATGTTTATTCAAAAGCGTTTGAGGATTTGCCTTTTTAGCTGTCGATAGCTTCTGGTTTAGCAATCATTTTGGTATGCCACACAGAGATAAATCGACCTTGGAAATTTAGACAAAAGTGGGATGAGTTGGCTATAATACCCGCTGCAGGTGTTCAAGTCGACAGGTCCGGTGTGGGGCATGCTCGTTGAGCATCAATCAATGCGGACGTGGTGTATTAAAGTCAATACATTAGGCTGACGCTTTTTATTTGTGGTTTGTAATCCACTCTTTTCAAATAGAATTCAGATCTAAGCGTTATTCAAATACGTCCCTTCACGTGGTGCACAACGCTCTAGCGATGGCCTCTGGGGTCAGCCACCATACAAAGGTGACGGTCTAGAACTCGGTCCGGGGGTTCCTTCATTGAGTGTATATCGGCATTAGCCACATCCGTTTTTTTTTATTGTAACCTTGAAGGGTATCTGTCATGCCGGTGACTTGACCCGAGTATCCGGTAAGTGCTCCCGGCCGCGTCTTGGCGCTCTGCTTATCGCTCTCGACGTTAGTTTACTCGGGCTTAGGAGTATCCAGTTCCTACGTGACTTTTAGACTAATTCTTTGCCTAGATGAGATCTAATTGATGATATGGAGAAAGTTAGTATTCGATATCCTGCCATGCAGGAACGACCTCTCCGTACCGATTAGAAGAGAACACACTCAATTAGCATCATATTTGGTTGTCAGATGATTTATGTCTTATTCCTCTTAATGTCCCTATAATTTAGGGTGACTCTTTATCCCAAGTCCGTTTTCAGCCTTTCTTATTTATTGCGGTAAGTTGTTTTGGTGAACAAAAATAAAAACTACCCTTTAACGGTTGTAGATTTAATATTTTACTGGAGACACAGTCTTTTATCAAGGTCCTTTGG

## Задача №3 (4)
Напишите алгоритм Витерби для восстановления последовательности скрытых состояний марковской модели из второго задаания. Воспользуйтесь им, воссстановив состояния тех последовательностей, которые вы получили во втором задании и посчитайте TP, TN, FP, FN по количеству правильно или ошибочно предсказанных позиций из CpG остравков. 

In [284]:
class Viterbi():
    def __init__(self, sequence, states, transition_probs, emission_probs, init_states_probs):
        self.sequence = sequence
        self.states = states
        self.transition_probs = transition_probs
        self.emission_probs = emission_probs
        self.init_states_probs = init_states_probs
        self.n_states = len(transition_probs[0])

    def viterbi(self):
        len_seq = len(self.sequence)

        # вероятность, что на предыдущем шаге находимся в состоянии self.alphabet_states[i]
        prev_state_prob = np.full(self.n_states, -np.inf)
        # сохраняем путь для каждого из состояний
        prev_path = np.zeros((self.n_states, len_seq), dtype='int')

        # заполняем для начальных состояний
        for j in range(self.n_states):
            prev_state_prob[j] = np.log(self.init_states_probs[j]) + np.log(self.emission_probs[j][self.sequence[0]])
            prev_path[j, 0] = j

        for i in range(1, len_seq):
            curr_state_prob = np.full(self.n_states, -np.inf)
            curr_path = np.zeros((self.n_states, len_seq), dtype='int')

            for j in range(self.n_states):  # текущее состояние
                max_prob = -np.inf
                # индекс наиболее вероятного состояния предыдущего шага для текущего состояния
                prev_state = 0
                for k in range(self.n_states):  # предыдущее состояние
                    prob = prev_state_prob[k] + np.log(self.transition_probs[k][j]) + np.log(
                        self.emission_probs[j][self.sequence[i]])
                    if prob > max_prob:
                        max_prob = prob
                        prev_state = k
                        
                curr_state_prob[j] = max_prob
                # копирование пути, который был до лучшего предыдущего состояния
                curr_path[j, :i] = prev_path[prev_state, :i]
                curr_path[j, i] = j

            prev_state_prob = curr_state_prob
            prev_path = curr_path

        final_state = np.argmax(prev_state_prob)
        return prev_path[final_state]

    def eval_viterbi(self):
        restored_states = self.viterbi()
        TP, TN, FP, FN = 0, 0, 0, 0
        for i in range(len(self.sequence)):
            if restored_states[i] == 0 and self.states[i] == '+':
                TP += 1
            elif restored_states[i] == 0 and self.states[i] == '-':
                FP += 1
            elif restored_states[i] == 1 and self.states[i] == '-':
                TN += 1
            elif restored_states[i] == 1 and self.states[i] == '+':
                FN += 1
        width = 10
        return f'TP: {str(TP).ljust(width)}TN: {str(TN).ljust(width)}FP: {str(FP).ljust(width)}FN: {FN}\n'

In [285]:
transition_probs = np.array([[0.95, 0.05], [0.005, 0.995]])  # + -
emission_probs = [islands_nucl, nonIslands_nucl]
init_states_probs = [0.1, 0.9]

После знака ">" записана длина последовательности.

TP - правильно предсказанное состояние '+', то есть состояние нахождения внутри CpG-островка.  
FP - правильно предсказанное состояние '-', то есть состояние нахождения вне CpG-островка.

In [286]:
with open(filename_generated_seq, 'r') as f:
    while True:
        header = f.readline().strip()
        if not header:
            break
            
        sequence = f.readline().strip()
        states = f.readline().strip()
        _ = f.readline()

        print(header)
        print(Viterbi(sequence, states, transition_probs, emission_probs, init_states_probs).eval_viterbi())

>1000
TP: 0         TN: 935       FP: 0         FN: 65

>5950
TP: 0         TN: 5298      FP: 0         FN: 652

>10900
TP: 78        TN: 9693      FP: 1         FN: 1128

>15850
TP: 258       TN: 13788     FP: 33        FN: 1771

>20800
TP: 155       TN: 18570     FP: 17        FN: 2058

>25750
TP: 115       TN: 23616     FP: 7         FN: 2012

>30700
TP: 116       TN: 28430     FP: 2         FN: 2152

>35650
TP: 313       TN: 32343     FP: 197       FN: 2797

>40600
TP: 68        TN: 37169     FP: 2         FN: 3361

>45550
TP: 99        TN: 40889     FP: 8         FN: 4554

>50500
TP: 371       TN: 45711     FP: 91        FN: 4327

>55450
TP: 504       TN: 50219     FP: 59        FN: 4668

>60400
TP: 571       TN: 54566     FP: 145       FN: 5118

>65350
TP: 268       TN: 59983     FP: 73        FN: 5026

>70300
TP: 360       TN: 64459     FP: 83        FN: 5398

>75250
TP: 157       TN: 68187     FP: 51        FN: 6855

>80200
TP: 305       TN: 73475     FP: 8         FN: 6412

>8

## Задача №4 (4)
Напишите алгоритм вперед назад для модели из второго задания. Пользуясь этим алгоритмом найдите вероятности того, что модель находилась в состоянии **+** для каждой позиции строк из второго задания. Устанавливая различные пороговые значения, определите позиции соответствующие CpG островкам и посчитайте TP. Постройте график зависимости TP от выбранного порогового значения. Есть ли пороговые значения при которых TP больше чем в задании №3?